In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import timedelta
import os
import gc
import csv

In [2]:
#日付列の作成
date_list = pd.date_range('2013/01/01', '2016/6/30', freq='D').strftime('%Y/%m/%d')
date_list = np.array([d for d in date_list])

In [3]:
from sklearn.utils.extmath import cartesian
import random
import itertools

# 店舗ID(8桁)と商品ID(10桁)の乱数データ作成
store_list = np.array(random.sample(range(10000000, 99999999, 1), k=30))
goods_list = np.array(random.sample(range(1000000000, 9999999999, 1), k=370))

# 日付、店舗ID, 商品IDの乱数データ生成
data = cartesian((date_list,store_list, goods_list))
df = pd.DataFrame(data, columns=['日付', '店舗ID', '商品ID'])

In [4]:
del date_list, store_list, goods_list
gc.collect()

20

In [5]:
#売上高、　販売数量の追加
rng = np.random.default_rng(42)
df['売上高'] = rng.integers(100,10000, size=len(df))
df['販売数量'] = rng.integers(1,100, size=len(df))

In [6]:
df.head(30)

,日付,店舗ID,商品ID,売上高,販売数量
0,2013/01/01,69918662,4518425933,983,19
1,2013/01/01,69918662,1689309740,7762,49
2,2013/01/01,69918662,8736268573,6580,24
3,2013/01/01,69918662,3810945780,4444,95
4,2013/01/01,69918662,5338891503,4386,84
5,2013/01/01,69918662,4474709459,8600,38
6,2013/01/01,69918662,7804297965,950,95
7,2013/01/01,69918662,4311011238,7003,61
8,2013/01/01,69918662,6091703643,2094,11
9,2013/01/01,69918662,5745948354,1032,19


In [7]:
df.tail(30)

,日付,店舗ID,商品ID,売上高,販売数量
14174670,2016/06/30,73397826,2718161570,6181,2
14174671,2016/06/30,73397826,9747050801,8085,24
14174672,2016/06/30,73397826,8657111082,5415,26
14174673,2016/06/30,73397826,6295935812,795,99
14174674,2016/06/30,73397826,7853063432,9277,11
14174675,2016/06/30,73397826,6388675544,918,61
14174676,2016/06/30,73397826,6509349034,8894,22
14174677,2016/06/30,73397826,7425402798,8429,18
14174678,2016/06/30,73397826,8617450767,3378,94
14174679,2016/06/30,73397826,5078762494,1674,17


In [8]:
# csvの保存
csv_path = '../data/TestData.csv'

if os.path.isfile(csv_path):
    os.remove(csv_path)

file = open(csv_path, 'w')
writer = csv.writer(file)
writer.writerow([c for c in df.columns])

for row in df.itertuples():
    writer.writerow(row[1:6])

# ファイルサイズの確認
print(os.path.getsize(csv_path))

578584473


In [9]:
import dask.dataframe as dd

# テストデータの読込
df = dd.read_csv(csv_path, encoding='shift jis').compute()
df.head(30)

,日付,店舗ID,商品ID,売上高,販売数量
0,2013/01/01,69918662,4518425933,983,19
1,2013/01/01,69918662,1689309740,7762,49
2,2013/01/01,69918662,8736268573,6580,24
3,2013/01/01,69918662,3810945780,4444,95
4,2013/01/01,69918662,5338891503,4386,84
5,2013/01/01,69918662,4474709459,8600,38
6,2013/01/01,69918662,7804297965,950,95
7,2013/01/01,69918662,4311011238,7003,61
8,2013/01/01,69918662,6091703643,2094,11
9,2013/01/01,69918662,5745948354,1032,19


In [10]:
# 日付をdatetime型に変換
df['日付（月）'] = [str[:7] + '/01' for str in df['日付']]

In [11]:
# 2. 1 月別、店別、総売上の集計
store_m_data = df.groupby(['日付（月）','店舗ID'], as_index=False).sum()[['日付（月）','店舗ID','売上高']]
# 日付、店番でのソート
store_m_data.sort_values(['店舗ID', '日付（月）'],  ignore_index=True, inplace=True)
# 列名のリネーム
store_m_data.rename(columns={'売上高':'総売上高'}, inplace=True)

In [12]:
# 2. 1 月別、店別、総売上の集計データの確認
store_m_data

,日付（月）,店舗ID,総売上高
0,2013/01/01,12285821,58295413
1,2013/02/01,12285821,52525059
2,2013/03/01,12285821,57984939
3,2013/04/01,12285821,56234545
4,2013/05/01,12285821,57571147
...,...,...,...
1255,2016/02/01,92301779,54084408
1256,2016/03/01,92301779,57557475
1257,2016/04/01,92301779,56069861
1258,2016/05/01,92301779,57234207


In [13]:
# 2. 2 月別、商品別、総売上の集計
goods_m_data = df.groupby(['日付（月）','商品ID'], as_index=False).sum()[['日付（月）','商品ID','売上高']]
goods_m_data.sort_values(['商品ID', '日付（月）'],  ignore_index=True, inplace=True)
goods_m_data.rename(columns={'売上高':'総売上高'}, inplace=True)

In [14]:
# 2. 2 月別、商品別、総売上の集計データの確認
goods_m_data

,日付（月）,商品ID,総売上高
0,2013/01/01,1013088055,4822330
1,2013/02/01,1013088055,4286450
2,2013/03/01,1013088055,4635797
3,2013/04/01,1013088055,4666614
4,2013/05/01,1013088055,4815584
...,...,...,...
15535,2016/02/01,9985866592,4268188
15536,2016/03/01,9985866592,4642944
15537,2016/04/01,9985866592,4604896
15538,2016/05/01,9985866592,4725722


In [15]:
# 2. 3 日別、商品別、平均価格の集計

# 小数点以下10桁に設定
pd.options.display.precision = 10

#単価の計算
df['単価'] = df['売上高'] / df['販売数量']

# 平均価格の集計
goods_d_data = df.groupby(['日付','商品ID'], as_index=False).mean()[['日付','商品ID','単価']]
goods_d_data.sort_values(['商品ID', '日付'],  ignore_index=True, inplace=True)
goods_d_data.rename(columns={'日付':'日付（日）','単価':'平均価格'}, inplace=True)

In [16]:
# 2. 3 日別、商品別、平均価格の集計
goods_d_data

,日付（日）,商品ID,平均価格
0,2013/01/01,1013088055,664.5569304402
1,2013/01/02,1013088055,277.0844449556
2,2013/01/03,1013088055,319.3881036351
3,2013/01/04,1013088055,210.9905544941
4,2013/01/05,1013088055,698.4436483856
...,...,...,...
472485,2016/06/26,9985866592,360.5064936656
472486,2016/06/27,9985866592,132.0142887196
472487,2016/06/28,9985866592,174.7259860315
472488,2016/06/29,9985866592,242.2915481832
